## Deferred Start Swap
 Deferred Start Swap은 스왑 계약이 체결된 후 일정 기간이 지난 후에 시작되는 스왑을 의미합니다. 예를 들어, 계약이 체결된 시점에서 1년 후에 시작하는 스왑을 생각해 볼 수 있습니다.

금리 외에도 다양한 자산을 기초 자산으로 하는 스왑을 구현할 수 있습니다. 예를 들어, 주식, 상품, 외환 등의 자산을 기초 자산으로 하는 스왑을 구현할 수 있습니다. 여기서는 주식을 기초 자산으로 하는 Deferred Start Swap을 예로 들어 설명해 보겠습니다.

주식을 기초 자산으로 하는 Deferred Start Swap 구현
주식을 기초 자산으로 하는 스왑에서는 두 당사자가 주식의 가격 변동에 따라 일정 기간 동안 지불을 교환합니다. Deferred Start Swap에서는 스왑이 일정 기간 후에 시작됩니다.

In [2]:
import matplotlib.font_manager as fm

# Google Colab에 NanumGothic 폰트 설치
!apt-get update -qq
!apt-get install fonts-nanum* -qq

# 설치된 폰트 확인 및 matplotlib에 적용
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font = fm.FontProperties(fname=font_path)
fm.fontManager.addfont(font_path)
plt.rc('font', family='NanumGothic')

Selecting previously unselected package fonts-nanum.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-3_all.deb ...
Unpacking fonts-nanum-coding (2.5-3) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-7_all.deb ...
Unpacking fonts-nanum-eco (1.000-7) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20200506-1_all.deb ...
Unpacking fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Setting up fonts-nanum-coding (2.5-3) ...
Setting up fonts-nanum-eco (1.000-7) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [1]:
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# 고정 금액 지불 계산 함수 (주식의 경우 일정한 금액을 지불하는 것으로 가정)
def fixed_leg_payment(notional, fixed_payment, payment_dates, start_date):
    payments = fixed_payment * np.diff(payment_dates, prepend=start_date)
    return payments

# 주식 가격 변동에 따른 변동 금액 지불 계산 함수
def equity_leg_payment(notional, stock_prices, payment_dates, start_date):
    payments = notional * np.diff(stock_prices, prepend=stock_prices[0])
    return payments

# Deferred Start Swap의 현재 가치 계산 함수
def deferred_start_swap_value(notional, fixed_payment, stock_prices, start_date, payment_dates, discount_factors):
    # 고정 금액 지불 계산
    fixed_leg_payments = fixed_leg_payment(notional, fixed_payment, payment_dates, start_date)

    # 주식 가격 변동에 따른 변동 금액 지불 계산
    equity_leg_payments = equity_leg_payment(notional, stock_prices, payment_dates, start_date)

    # 스왑의 현재 가치 계산
    swap_value = np.sum((equity_leg_payments - fixed_leg_payments) * discount_factors)

    return swap_value

# 실제 데이터 가져오기
# Apple Inc. 주가 데이터
apple = yf.Ticker("AAPL")
apple_data = apple.history(period="1y")
S_values = apple_data['Close'].values

# 기본 변수 설정
notional = 1000  # 명목 금액
fixed_payment = 5  # 고정 지불 금액
start_date = 1  # 스왑 시작 시점 (1년 후)
payment_dates = np.array([2, 3, 4, 5, 6])  # 지불 날짜 (스왑 시작 후 매년)
discount_factors = np.exp(-0.05 * (payment_dates - start_date))  # 할인율

# 주가 데이터 (매년 말 주가로 가정)
annual_stock_prices = S_values[::len(S_values)//len(payment_dates)]

# Deferred Start Swap의 현재 가치 계산
swap_value = deferred_start_swap_value(notional, fixed_payment, annual_stock_prices, start_date, payment_dates, discount_factors)
print(f"Deferred Start Swap의 현재 가치: {swap_value:.2f}")

# 시각화
plt.figure(figsize=(10, 6))
plt.plot(payment_dates, fixed_leg_payment(notional, fixed_payment, payment_dates, start_date), label='고정 금액 지불')
plt.plot(payment_dates, equity_leg_payment(notional, annual_stock_prices, payment_dates, start_date), label='주식 가격 변동에 따른 지불')
plt.xlabel('지불 날짜 (년)')
plt.ylabel('지불 금액')
plt.title('Deferred Start Swap 지불 내역')
plt.legend()
plt.grid(True)
plt.show()


ValueError: operands could not be broadcast together with shapes (6,) (5,) 